In [1]:
from llama_index import Document, VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index.storage.storage_context import StorageContext
# from transformers import AutoTokenizer, AutoModel
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
from llama_index import Document, VectorStoreIndex 
from llama_index.node_parser import SentenceSplitter
import chromadb
import pandas as pd 
import openai
import re
import os
import getpass
import glob 

In [2]:
default_path = os.getcwd()
model_path = os.path.join(default_path, '../../models')
model_dir = os.path.join(model_path, "mistral_origin")
data_path = os.path.join(default_path, '../../../data')
rulebook_path = os.path.join(data_path, 'pdf', 'rules')

In [3]:
file_list = glob.glob(f'{rulebook_path}/*.pdf')
file_list[6]

'/rag/jupyter/llama-index_examples/embedding/../../../data/pdf/rules/신여비교통비.pdf'

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key: ········


In [6]:
chroma_client = chromadb.HttpClient(host="192.168.0.146")
chroma_client.list_collections()

[Collection(name=deposit),
 Collection(name=data),
 Collection(name=card),
 Collection(name=loan)]

In [7]:
data_collection = chroma_client.get_or_create_collection("data")
data_collection

Collection(name=data)

In [8]:
data_store = ChromaVectorStore(chroma_collection=data_collection)
data_storage = StorageContext.from_defaults(vector_store=data_store)

In [90]:
documents = SimpleDirectoryReader(input_files=[file_list[8]]).load_data()

In [91]:
num_mapper = dict({'①':'제1항', '②':'제2항', '③':'제3항', '④':'제4항', '⑤':'제5항', '⑥':'제6항',\
                   '⑦':'제7항', '⑧': '제8항', '⑨': '제9항', '⑩': '제10항', '⑪': '제11항', '⑫': '제12항',\
                  '⑬':'제13항', '⑭':'제14항', '⑮':'제15항', '⑯':'제16항', '⑰':'제17항', '⑱':'제18항', '⑲':'제19항', '⑳':'제20항'})

In [92]:
def text_cleanse(document):
    '''
    여러 공백 문자열 단일 공백 문자로 변환 
    여러 줄변환 문자 줄변환 문자로 변환 (\n x 2~ => \n x 2) 
    문서 맨 윗 내용이 페이지 번호인 경우 페이지 번호 제거 
    '''
    document.text = re.sub('[^A-Za-z0-9\'\"\-가-힣(){}\\n[]]', '', document.text)
    for num in num_mapper.keys():
        document.text = document.text.replace(num, num_mapper[num])
    document.text = re.sub(' +', ' ', document.text)
    document.text = document.text.strip()
    # document.text = re.sub(r'\r\n{2,}', '\n\n', document.text)
    
    text_list = document.text.splitlines(True)
    if text_list[0].startswith('페이지') or text_list[0].startswith(document.metadata['page_label']):
        text_list = text_list[1:]

    new_text = [] 
    for line in text_list:
        if line == '\n' or line == ' \n':
            continue 
        new_text.append(line) 
    document.text = ''.join(new_text)
    return document

In [93]:
for idx in range(len(documents)):
    documents[idx] = text_cleanse(documents[idx])

### Document Processing 

In [94]:
def get_start_point(documents):
    '''
    목차 등을 걸러내고 문서 본문이 시작되는 지점을 찾는 함수
    '''
    s_point = 1
    for doc in documents:
        if len(re.findall(r'제 *1 *장', doc.text)) != 0 and (len(re.findall(r'목 *차', doc.text)) == 0 and len(re.findall(r'차 *례', doc.text)) == 0): 
            s_point = doc.metadata['page_label']
            break 
    return int(s_point) - 1

In [95]:
s_point = get_start_point(documents)
s_point

1

In [96]:
doc = '오늘 하루도 잘 마무리되어 가고 있다. \n 내일 하루는 어떨까 ? \n 기대가 안되는건 아니다 \n\n'
text_list = doc.splitlines(True) 
text_list[:1], text_list[2:]

(['오늘 하루도 잘 마무리되어 가고 있다. \n'], [' 기대가 안되는건 아니다 \n', '\n'])

In [97]:
val_dict = dict()
len(val_dict)

0

In [ ]:
def split_doc(split_spots: dict, document):
    '''
    주어진 spot에 따라 문서를 분리하는 함수
    
    split_spots: {int: val, int: val, ...}
    e.g) split_spots: {1 (1번째 line): '제1장', 3 (3번째 line): '제2장'}
    len(split_spots): 2  => 3개의 document 획득
    '''
    text_list = document.text.splitlines(True)
    splitted_doc = [] 
    prev_spot = 0 
    
    for spot in split_spots.keys():
        print(f'spot: {spot}')
        prev_doc_texts = text_list[prev_spot:spot]
        prev_doc_text = ''.join(prev_doc_texts)
        prev_spot = spot + 1
        prev_doc = Document(text=prev_doc_text, 
                            doc_id=f'{file_name}_doc_{doc_no}',
                            metadata={'spot': split_spots[spot], 'file_name': file_name},
                            excluded_llm_metadata_key=['spot', 'file_name'])
        splitted_doc.append(prev_doc)
    return splitted_doc

In [ ]:
global file_name 

file_name = documents[0].metadata['file_name'].split('.')[0]
file_name

In [ ]:
def get_new_doc_content(document):  
    '''
    제1장 ~ 제n장 및 부칙을 문서 단위로 가져오는 함수 
    input: old_document (페이지 단위로 분할된 document object) 
    output: new_documents (장 단위로 분할된 document objects) 
    idx: 파일명 설정하기 위한 값, 페이지 번호 
    '''
    global prev_spot 
    # print(f'prev_spot: {prev_spot}')
    doc_list = []; meta_info = dict();
    text_list = document.text.splitlines(True)
    split_spots = dict() 
    print(text_list)
    for idx, text in enumerate(text_list):
        content_spots = re.findall(r'제 *[0-9] *장', text)
        extra_spots = re.findall(r'부 *칙', text)
        if len(content_spots) == 0 and extra_spots == 0:
            continue 
        elif len(content_spots) > 0:
            # print(f'content spot: {content_spots}')
            text_list.insert(idx, '\n')
            split_spots[idx] = content_spots
            prev_spot = content_spots
        elif len(extra_spots) > 0:
            # print(f'extra spot: {extra_spots}')
            split_spots[idx] = extra_spots 
            text_list.insert(idx, '\n')
            prev_spot = content_spots 

    if len(split_spots) == 0: 
        new_text = ''.join(text_list)
        doc = Document(text=new_text, 
                       doc_id=f'{file_name}_doc_{doc_no}',
                       metadata={'spot': prev_spot, 'file_name': file_name},
                       excluded_llm_metadata_keys=['spot', 'file_name']
              )
        return doc 
        
    splitted_docs = split_doc(split_spots, document)
    return splitted_docs

In [ ]:
global prev_spot
global doc_no

doc_no = 1 
prev_spot='제1장'

In [ ]:
s_point

In [ ]:
new_documents = []
for idx, doc in enumerate(documents):
    # print(idx)
    if idx >= s_point:
        new_documents.append(get_new_doc_content(doc))   # idx: 페이지 번호 

[' 윤 리 헌 장 \n', '우리는 기업의 사회적 책임을 소중히 여기고, 고객의 가치와 신뢰를 얻기 위해 \n', '비즈니스 활동에서 공정한 기준에 따라 행동하며 이러한 기준에서 벗어나는 \n', '어떠한 결과도 수용하지 않음을 선언한다 . \n', '우리가 지키는 높은 윤리수준은 \n', '명문화된 규정보다 임직원 각자의 자발적 참여로 구축되는 건전한 기업문화로 자연스럽게 달성되며 \n', '아름다운 전통으로 계승한다 . \n', '하나, 우리는 높은 윤리의식과 밝고 고운 심성을 바탕으로 정직하고 공정하게 직무를 수행한다 . \n', '하나, 우리는 최고의 솔루션과 서비스로 고객의 소중한 가치를 항상 최우선으로 생각한다 . \n', '하나, 우리는 효율적인 경영으로 이익을 실현함으로써 주주가치를 존중한다 . \n', '하나, 고객 및 협력사와의 거래는 상호 발전을 기함과 동시에 투명하고 공정하게 한다. \n', '하나, 우리는 국가와 지역사회의 각종 법규를 준수한다 . \n', '하나, 우리는 일체의 부정행위를 하지 않으며 업무수행 시 회사이익을 우선한다 . \n', '윤 리 강 령 \n', '제 1 장 기 본 책 무 \n', '제1조 (최고의 제품과 서비스창출 ) \n', '제1항 품질은 결코 타협하거나 양보할 대상이 아닌 우리의 최고 가치로 삼는다. \n', '제2항 가장 좋게, 싸게, 빠르게, 최고의 경쟁력을 확보하기 위해 최선을 다한다. \n', ' 제3항 부단한 혁신과 연구개발을 통해 새로운 가치를 창조함으로 써 인류사회의 발전을 선도한다 . \n', '제2조 (법규준수와 공정한 경쟁) \n', '제1항 사업활동에 있어서 모든 법규와 규범, 기초질서를 충실히 준수하며 , 문화와 관습을 존중한다 . \n', '제2항 경쟁사를 존중하며 , 정당한 방법과 실력으로 자유롭고 공정하게 경쟁한다 . \n', '제3항 공정한 거래를 위하여 자유경쟁의 원칙에 따라 경쟁한다 . \n', '제4항 공정거래 당사자들과는 상호 신뢰를 기반으로 공동발전을 도모한다 . \

In [ ]:
print(new_documents[0].text)